# Wide and deep neural networks

## Load diabetes dataset

In [1]:
# make sure we have our data file available
import os
os.popen('ls -l data/*').read().strip()
# filename = os.popen('ls -l data/*').read().strip().split('/')[1]

'-rw-r--r--  1 Neo  staff  53438 Mar 11 14:35 data/diabetes.csv'

In [3]:
print("number of records: ", os.popen('cat data/diabetes.csv | wc -l').read().strip())


number of records:  759


In [2]:
print("Data sample: ", "\n", os.popen('head -n 5 data/diabetes.csv').read().strip())

Data sample:  
 -0.294118,0.487437,0.180328,-0.292929,0,0.00149028,-0.53117,-0.0333333,0
-0.882353,-0.145729,0.0819672,-0.414141,0,-0.207153,-0.766866,-0.666667,1
-0.0588235,0.839196,0.0491803,0,0,-0.305514,-0.492741,-0.633333,0
-0.882353,-0.105528,0.0819672,-0.535354,-0.777778,-0.162444,-0.923997,0,1
0,0.376884,-0.344262,-0.292929,-0.602837,0.28465,0.887276,-0.6,0


In [4]:
# FIXME: use this piece of code to preview the data nicely
# import tabulate
# with open('data/diabetes.csv') as inf:
#     print(tabulate(inf, headers="firstrow"))

In [5]:
import numpy as np
xy = np.loadtxt('data/diabetes.csv', delimiter=',', dtype=np.float32)

In [6]:
import torch
from torch.autograd import Variable

In [7]:
x_data = Variable(torch.Tensor(torch.from_numpy(xy[:, 0:-1])))
print("x_data shape: \n", x_data.size())
# print first 5 rows
print("first five rows: \n", x_data[0:5])

x_data shape: 
 torch.Size([759, 8])
first five rows: 
 tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])


In [8]:
y_data = Variable(torch.Tensor(torch.from_numpy(xy[:, [-1]])))
print("y_data shape: \n", y_data.size())
# print first 5 rows
print("first five rows: \n", y_data[0:5])

y_data shape: 
 torch.Size([759, 1])
first five rows: 
 tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


## Define Model

In [15]:
class DiabetesPredictionModel(torch.nn.Module):
    """Some Information about DiabetesPredictionModel"""
    def __init__(self):
        super(DiabetesPredictionModel, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        # Model architecture:
        # x----[l1]---[sig]---[l2]---[sig]---[l3]---[sig]---y_hat
        #                  |              | .             |
        #                 o1 .           o2 .            o3

        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))

        return y_pred

In [16]:
diabetes_prediction_model = DiabetesPredictionModel()

## Define Loss and Optimisation algorithm

In [21]:
criterion = torch.nn.BCELoss(size_average=True)

In [22]:
learning_rate = 0.1
optimiser = torch.optim.SGD(diabetes_prediction_model.parameters(), lr=learning_rate)

## Training loop

In [23]:
for epoch in range(100):
    # forward pass
    y_pred = diabetes_prediction_model.forward(x_data)

    # compute loss
    loss = criterion(y_pred, y_data)
    print(epoch, loss.data.item())

    # backpropagation
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

0 0.6450392007827759
1 0.645038902759552
2 0.6450377702713013
3 0.6450371146202087
4 0.6450362801551819
5 0.6450356841087341
6 0.6450350880622864
7 0.6450344920158386
8 0.6450337171554565
9 0.6450329422950745
10 0.6450326442718506
11 0.6450315713882446
12 0.6450310945510864
13 0.6450303792953491
14 0.6450300216674805
15 0.6450293064117432
16 0.6450287699699402
17 0.6450280547142029
18 0.6450275182723999
19 0.6450267434120178
20 0.6450260281562805
21 0.6450254917144775
22 0.6450248956680298
23 0.645024299621582
24 0.6450236439704895
25 0.645022451877594
26 0.6450213193893433
27 0.6450207829475403
28 0.6450202465057373
29 0.6450196504592896
30 0.6450187563896179
31 0.6450176239013672
32 0.6450172662734985
33 0.6450169086456299
34 0.6450163722038269
35 0.6450157761573792
36 0.6450150609016418
37 0.6450144052505493
38 0.645013689994812
39 0.6450128555297852
40 0.6450120806694031
41 0.645011305809021
42 0.6450103521347046
43 0.6450096368789673
44 0.6450091004371643
45 0.6450085043907166
46 